In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from pymc_marketing.mmm import MMM, GeometricAdstock, LogisticSaturation
from mmm_eval import PyMCConfig, run_evaluation

# Load your data

In [2]:
# Load your data - we have a revenue column and a response column (quantity)
data = pd.read_csv("data/example_data.csv")
data.head()

,date_week,quantity,price,revenue,radio,TV,event_1,event_2,dayofyear
0,2018-04-02,1250.138126,5.000322,6251.093528,31.858002,0.0,0.0,0.0,92
1,2018-04-09,1254.595052,5.015090,6291.906837,11.238848,0.0,0.0,0.0,99
2,2018-04-16,1300.522776,5.029466,6540.934884,29.240027,0.0,0.0,0.0,106
3,2018-04-23,1279.793864,5.043116,6454.149541,7.139855,0.0,0.0,0.0,113
4,2018-04-30,1242.244560,5.055754,6280.483221,38.674515,0.0,0.0,0.0,120


# Fit a PyMC MMM

In [3]:
# Fit a pymc model

X = data.drop(columns=["revenue","quantity"])
y = data["quantity"]

base_model = MMM(
    date_column="date_week" ,
    channel_columns=["TV","radio"],
    adstock=GeometricAdstock(l_max=4),
    saturation=LogisticSaturation()
)

base_model.fit(X=X, y=y, chains=4, target_accept=0.85)

Output()

There were 1 divergences after tuning. Increase `target_accept` or reparameterize.
2025-07-09 14:56:55,233 - pymc.stats.convergence - ERROR - There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data
	> fit_data

# I wonder if this MMM is any good... let's check!

In [4]:
# We just need to make a config using the model object and some fit kwargs

fit_kwargs = { 
    "chains": 4,
    "target_accept": 0.85,
}

config = PyMCConfig.from_model_object(base_model, fit_kwargs=fit_kwargs, response_column="quantity", revenue_column="revenue")

# Save this for later if you want to run from CLI!
config.save_model_object_to_json(save_path="data/", file_name="saved_config")

PyMCConfig(revenue_column='revenue', response_column='quantity', pymc_model_config=PyMCModelSchema(date_column='date_week', channel_columns=['TV', 'radio'], adstock=GeometricAdstock(prefix='adstock', l_max=4, normalize=True, mode='After', priors={'alpha': Prior("Beta", alpha=1, beta=3, dims="channel")}), saturation=LogisticSaturation(prefix='saturation', priors={'lam': Prior("Gamma", alpha=3, beta=1, dims="channel"), 'beta': Prior("HalfNormal", sigma=2, dims="channel")}), time_varying_intercept=False, time_varying_media=False, sampler_config={}, validate_data=True, control_columns=None, yearly_seasonality=None, adstock_first=True, dag=None, treatment_nodes=None, outcome_node=None), fit_config=PyMCFitSchema(draws=None, tune=None, chains=4, target_accept=0.85, random_seed=None, progressbar=False, return_inferencedata=True), date_column='date_week', channel_columns=['TV', 'radio'], control_columns=None)

In [6]:
# Run the evaluation suite!
result = run_evaluation(framework="pymc_marketing", config=config, data=data)

2025-07-09 15:00:26,950 - mmm_eval.core.validation_test_orchestrator - INFO - Running test: accuracy
2025-07-09 15:00:26,951 - mmm_eval.core.base_validation_test - INFO - Splitting data into train and test sets for accuracy test
2025-07-09 15:00:33,766 - mmm_eval.core.validation_tests - INFO - Saving the test results for accuracy test
2025-07-09 15:00:33,766 - mmm_eval.core.validation_test_orchestrator - INFO - Running test: cross_validation
2025-07-09 15:00:33,767 - mmm_eval.core.base_validation_test - INFO - Splitting data into train and test sets for cross_validation test
2025-07-09 15:00:33,769 - mmm_eval.core.validation_tests - INFO - Running cross-validation fold 1 of 5
There were 4 divergences after tuning. Increase `target_accept` or reparameterize.
2025-07-09 15:00:40,625 - pymc.stats.convergence - ERROR - There were 4 divergences after tuning. Increase `target_accept` or reparameterize.
2025-07-09 15:00:41,003 - mmm_eval.core.validation_tests - INFO - Running cross-validation

# Examine the results

In [8]:
# Let's see what we got
display(result)

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,mape,mape,0.063775,True,accuracy,2025-07-09T15:00:33.766675
1,r_squared,r_squared,-0.007427,False,accuracy,2025-07-09T15:00:33.766675
2,mean_mape,mean_mape,0.084227,True,cross_validation,2025-07-09T15:01:08.600905
3,std_mape,std_mape,0.058306,False,cross_validation,2025-07-09T15:01:08.600905
4,mean_r_squared,mean_r_squared,-7.051987,False,cross_validation,2025-07-09T15:01:08.600905
5,mean_percentage_change,mean_percentage_change_TV,0.021199,True,refresh_stability,2025-07-09T15:02:16.867374
6,mean_percentage_change,mean_percentage_change_radio,0.467239,False,refresh_stability,2025-07-09T15:02:16.867374
7,std_percentage_change,std_percentage_change_TV,0.021548,True,refresh_stability,2025-07-09T15:02:16.867374
8,std_percentage_change,std_percentage_change_radio,0.700347,False,refresh_stability,2025-07-09T15:02:16.867374
9,percentage_change,percentage_change_TV,0.008652,True,perturbation,2025-07-09T15:02:30.765531
